# Getting batch prediction EDA with Tensor Board and Text Gecko

In [ ]:
import pandas as pd
from typing import Dict
from datasets import load_dataset
from typing import Dict, List
import tensorflow as tf
import numpy as np
import os
from tensorboard.plugins import projector

LIMIT = 3000
PROJECT_ID = 'wortz-project-352116'
DATASET = 'ecomm-embedding'
BUCKET = 'gs://ecomm-query-product-pairs'
USER_PROMPT = 'User query: '
PRODUCT_PROMPT = 'Product title: '

#### Load the data from huggingface

In [ ]:
raw_data = load_dataset("tasksource/esci")

In [ ]:
def get_input_dataframe(
    raw_data: Dict,
    user_prompt: str = USER_PROMPT,
    product_prompt: str = PRODUCT_PROMPT,
    limit: int = LIMIT,
) -> pd.DataFrame:
    """
    This function returns batch prediction data for embeddings
    """

    for i, row in enumerate(raw_data["train"]):
        if i == limit - 1:
            break
        elif i == 0:
            query_prod_pairs = pd.DataFrame(
                {"content": [f'{user_prompt}{row["query"]}'], "type": ["query"]}
            )
        else:
            query_prod_pairs = pd.concat(
                [
                    query_prod_pairs,
                    pd.DataFrame(
                        {"content": [f'{user_prompt}{row["query"]}'], "type": ["query"]}
                    ),
                ]
            )
        query_prod_pairs = pd.concat(
            [
                query_prod_pairs,
                pd.DataFrame(
                    {
                        "content": [f'{product_prompt}{row["product_title"]}'],
                        "type": ["product_title"],
                    }
                ),
            ]
        )
    return query_prod_pairs

In [ ]:
query_prod_pairs = get_input_dataframe(raw_data)

In [ ]:
query_prod_pairs = query_prod_pairs.reset_index()
query_prod_pairs.head()

# Batch prediction

https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/batch-prediction-genai-embeddings#request_a_batch_response

____

In [ ]:
! gsutil mb $BUCKET

In [ ]:
output_file = 'batch_prediction_inputs.jsonl'

with open(output_file, 'w') as f:
    f.write(query_prod_pairs[['content']].to_json(lines=True, orient='records'))

In [ ]:
! gsutil cp $output_file $BUCKET

In [ ]:
from datetime import datetime
now = datetime.now()
now_string_tag = now.strftime("%Y-%m-%d-%H-%M-%S")
print("Tag for this run: ", now_string_tag)

In [ ]:
from vertexai.preview.language_models import TextEmbeddingModel

textembedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko")
batch_prediction_job = textembedding_model.batch_predict(
    dataset=[f"{BUCKET}/{output_file}"],
    destination_uri_prefix=f"{BUCKET}/batch-predict-{now_string_tag}",
)
print(batch_prediction_job.display_name)
print(batch_prediction_job.resource_name)
print(batch_prediction_job.state)

#### When complete you should see something like this

<img src='../img/bp-job.png' width=600px />

<img src='../img/output-data.png' width=600px />

### Visualize the embeddings with Tensorboard

Following this guide https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin

In [ ]:
bp_output_gcs_folder = batch_prediction_job.output_info.gcs_output_directory

! gsutil cp $bp_output_gcs_folder/* .

In [ ]:
predictions = pd.read_json(path_or_buf='000000000000.jsonl', lines=True)
predictions.head()

In [ ]:
def get_predictions(df: pd.DataFrame) -> List[List[float]]:
    embedding_list = []
    for _, row in df.iterrows():
        single_emb = row['predictions'][0]['embeddings']['values']
        embedding_list.append(single_emb)
    return embedding_list

embedding_list = get_predictions(predictions)
len(embedding_list)

In [ ]:
# Set up a logs directory, so Tensorboard knows where to look for files.
log_dir = "logs/ecomm-example/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, "metadata.tsv"), "w") as f:
    for instance in predictions.instance:
        data_type = instance['content'].split(': ')[0]
        # data_type = data_type
        data = ''.join(instance['content'].split(': ')[1:])
        f.write(f"{data_type}    {data}\n")


# Save the weights we want to analyze as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, here
# we will remove this value.
weights = tf.Variable(embedding_list)
# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = "metadata.tsv"
projector.visualize_embeddings(log_dir, config)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Now run tensorboard against on log data we just saved.
%tensorboard --logdir logs/ecomm-example/

#### The above will run until you stop it

You should be able to investigate the embedding space via PCA. Note the total variance captured to understand how complete the veiw investigate

<img src="../img/tensorboard.png" width=600px />


#### Also a great way to understand performance is to select a point of interest and top k neighbors appear

Below, we see natural hair dye query and it's associated nearest product description in the embedding space:


<img src="../img/knn-analysis.png" width=900px />


